# FINAL PROJECT: BOOTSTRAPPING

### Name: Carlos Cocha

#### Loading the required packages

In [1]:
import statistics
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [14]:
def average(x, sx = [], w = False):
    '''
    Function to calculate the average (weighted if w = True) of a set x.
    
    Parameters:
    x     :: set of data
    sx    :: uncertainty of the data
    
    Example:
    average([2,5,4,5,7], [0.5,0.2,0.4,0.5,0.3],  w = True)
    - 5.07887
    '''
    if w == True:
        n, d  = 0, 0
        for i in range(len(x)):
            n += x[i]/(sx[i]*sx[i])
            d += 1/(sx[i]**2)
        x_w = n/d
    else:
        x_w = np.sum(x)/len(x)
    return x_w

In [53]:
def slope_inter(x, y, w = False, ws = [] ):
    '''
    Function to calculate the intercept (y0) and the slope (m) parameters of
    y = y0 + m*x or y = y0 + m*[x-x^](if w = True)  given  2 sets of data (x, y). 
    
    Parameter:
    x :: set of data x
    y :: set of data y
    
    Example:
    slope_inter([0.5 , 0.9 , 1.5 , 1.8 , 2.65], [2.4 , 3.1 , 4.7 , 4.9 , 6.3])
    (4.279999999999999, 1.8414017341040463)
    '''
    numerator = 0
    denominator = 0
    
    if w == True:
        x_m =  average(x, sy, w = False)
        y_m = average(y, sy, w = False)

        numerator = np.sum(ws*(x-x_m)*(y-y_m))
        denominator = np.sum(ws*(x-x_m)*(x-x_m))
        m = numerator/denominator #slope
        y0 = y_m                  #interception
        
    else:
        x_m = average(x)
        y_m = average(y)
        
        numerator = np.sum((x-x_m)*(y-y_m))
        denominator = np.sum((x-x_m)*(x-x_m))
        m = numerator/denominator  #slope
        y0 = y_m - m*x_m           #interception
    
    return y0 , m

In [37]:
def cov(array1, array2):
    '''
    Function to calculate de covariance of 2 datasets.
    
    Parameters:
    array1 :: set of data1
    array2 :: set of data2
    
    Example:
    cov([1, 2, 4, 6, 3], [3, 5, 3, 2, 7])
    -1.5    
    '''
    n = 0
    M = len(array1)
    a1_m = np.array(array1).mean()
    a2_m = np.array(array2).mean()
    for i in range(M):
        n += (array1[i] - a1_m)*(array2[i] - a2_m)
    covariance = n/(M-1)
    return covariance

In [38]:
def stand_d(x):
    '''
    Function to calculate the standard deviation of a set x.
    
    Parameters:
    x :: set of data
    
    Example:
    stand_d([2,5,2,4,5,7,3])
    - 1.82574
    '''
    x_m = np.mean(x)
    n = 0
    for i in range(len(x)):
        n += (x[i] - x_m)**2
    sd = np.sqrt(n/(len(x) - 1))
    return sd

In [49]:
def pearson(x, y, w = False, ws = []):
    '''
    Function to calculate the Pearson’s r coefficient given x and y data.
    
    Parameters:
    x :: set of data
    y :: set of data
    ws = weights 
    
    Example:
    pearson(x, y, w = True, ws = 1/sy**2)
    0.99463
    '''
    if w == True:
        n = np.sum(ws)*np.sum(ws*x*y)-np.sum(ws*x)*np.sum(ws*y)
        d = np.sqrt(np.sum(ws)*np.sum(ws*x*x)-(np.sum(ws*x))**2)*np.sqrt(np.sum(ws)*np.sum(ws*y*y)-(np.sum(ws*y))**2)
        r = n/d
    else:
        sx = np.sqrt(np.sum((x-average(x))**2)/(len(x)-1))
        sy = np.sqrt(np.sum((y-average(y))**2)/(len(y)-1))
        n = cov(x, y)
        d = sx*sy
        r = n/d
    return r    

### 1. Read the data: x𝑖 ± 𝜎x𝑖, y𝑖 ± 𝜎y𝑖

In [2]:
# Reading the data from a .csv file
data = pd.read_csv("DATA.csv")
headers = ['x','y','sigma_x','sigma_y']
dframe = pd.DataFrame(data.values, columns = headers)
dframe

,x,y,sigma_x,sigma_y
0,0.50,2.40,0.02,0.6
1,0.90,3.10,0.09,1.8
2,1.50,4.70,0.15,1.5
3,1.80,4.90,0.18,0.6
4,2.65,6.30,0.45,3.0
5,3.00,8.05,0.05,0.2
6,3.45,8.80,0.07,0.4
7,4.10,11.50,0.25,3.2
8,4.65,11.20,0.15,0.8
9,5.25,14.30,0.25,2.4


In [3]:
# Copying the data into variables 
x = dframe['x']
y = dframe['y']
sx = dframe['sigma_x']
sy = dframe['sigma_y']

### 2. Obtain up to M = 1000 sets of datapoints {{𝑥𝑖, 𝑦𝑖} k} 𝑖  = 1, …, 𝑁,  𝑘 = 1, …, 𝑀 by resampling from your experimental data. Use the generator producing random numbers with Gaussian distribution

In [50]:
M = 1000
resampling = [] # to save all the resampling data
for i in range(M):
    x_d , y_d = [] , []
    for j in range(len(x)):
        # random.randn produce random numbers with Gaussian distribution
        x_d.append(x[j] + sx[j]*np.random.randn())
        y_d.append(y[j] + sy[j]*np.random.randn())
    resampling.append([x_d, y_d])

### 3. For each artificially resampled dataset, calculate their own slope and intercept (do NOT calculate their errors here!).

In [55]:
y0_m_dat = np.zeros((M, 2))  
y0_mw_dat = np.zeros((M, 2)) # to save the weighted fitting parameters
for i in range(M):
    # Calculate the slope and interception of each resampling dataset.
    y0_m_dat[i][0], y0_m_dat[i][1] = slope_inter(resampling[i][0], resampling[i][1])
    # For the weighted fitting w = True
    y0_mw_dat[i][0], y0_mw_dat[i][1] = slope_inter(resampling[i][0], resampling[i][1],  w = True, ws = 1/(sy**2))